In [1]:
import numpy as np
import os

from neuron import h
from neuron.units import mV, ms
h.load_file("stdrun.hoc")
h.load_file("stdlib.hoc")
h.load_file("import3d.hoc")

import json

import sys
sys.path.insert(1, "../../utils/")
import Stimuli, Pyr, HH

## generate cell

In [2]:
stim_locs_file = '../../data/morpho/segment_arrays/segment_array_8.5Hz_ID17.npy'
history_file = '../../data/morpho/random_histories/history_2_ID17.json'

duration = 1000

pyr = Pyr.Pyr()

## connect stimuli

In [3]:
stim_params = Stimuli.ExperimentalStimParams()
stim_scaffold = stim_params.stim_scaffold['pyr']

stim_locs = np.load(stim_locs_file)

stimuli = Stimuli.MorphoStimuli(
    f'stimset',
    stim_scaffold['stim_type_array'],
    stim_locs, stim_scaffold,
    duration
)

pyr.add_stimuli(stimuli)

In [ ]:
# check stimuli locations

## initialize state variables

In [4]:
pyr.initialize_state_vars(history_file)

In [ ]:
# check state variable initializations
with open(history_file) as fin:
    history = json.load(fin)

In [ ]:
seg_history

In [ ]:
def history_walk(seg_history, secs):
    for key, item in seg_history.items():
        if type(item) is dict:
            if len(item.keys()) > 0:
                new_seg_history = item
                new_secs = [getattr(sec, str(key)) for sec in secs]
                history_walk(new_seg_history, new_secs)
        else:
            for val, sec in zip(item, secs):
                setattr(sec, key, val)
        
history_walk(seg_history, secs)

In [ ]:
secs[-1].cagk.o

In [ ]:
secs[-2].cagk.o

In [ ]:
secs[-1].can.h

In [ ]:
secs[-1].na3.s

In [ ]:
secs[-1].v

In [ ]:
seg_history

## run simulation

In [ ]:
h.celsius = 35
h.finitialize()
h.continuerun(duration)

In [ ]:
list(pyr._vs[0])